### Курсовой проект по курсу: Библиотеки Python для Data Science. Продоолжение

#### Задача

Требуется, на основании имеющихся данных о клиентах банка, построить модель, используя обучающий датасет, для прогнозирования невыполнения долговых обязательств по текущему кредиту. Выполнить прогноз для примеров из тестового датасета.

#### Метрика качества

F1-score (sklearn.metrics.f1_score)

#### Требования к решению

Целевая метрика: F1 > 0.5 Метрика оценивается по качеству прогноза для главного класса (1 - просрочка по кредиту)

#### Решение должно содержать

- Тетрадка Jupyter Notebook с кодом Вашего решения, названная по образцу {ФИО}_solution.ipynb, пример SShirkin_solution.ipynb
- Файл CSV с прогнозами целевой переменной для тестового датасета, названный по образцу {ФИО}_predictions.csv, пример SShirkin_predictions.csv
- Рекомендации для файла с кодом (ipynb)
- Файл должен содержать заголовки и комментарии (markdown)
- Повторяющиеся операции лучше оформлять в виде функций
- Не делать вывод большого количества строк таблиц (5-10 достаточно)
- По возможности добавлять графики, описывающие данные (около 3-5)
- Добавлять только лучшую модель, то есть не включать в код все варианты решения проекта
- Скрипт проекта должен отрабатывать от начала и до конца (от загрузки данных до выгрузки предсказаний)
- Весь проект должен быть в одном скрипте (файл ipynb).
- Допускается применение библиотек Python и моделей машинного обучения, которые были в данном курсе.

#### Сроки сдачи
Cдать проект нужно в течение 5 дней после окончания последнего вебинара. Оценки работ, сданных до дедлайна, будут представлены в виде рейтинга, ранжированного по заданной метрике качества. Проекты, сданные после дедлайна или сданные повторно, не попадают в рейтинг, но можно будет узнать результат.

#### Примерное описание этапов выполнения курсового проекта
- Построение модели классификации
- Обзор обучающего датасета
- Обработка выбросов
- Обработка пропусков
- Анализ данных
- Отбор признаков
- Балансировка классов
- Подбор моделей, получение бейзлана
- Выбор наилучшей модели, настройка гиперпараметров
- Проверка качества, борьба с переобучением
- Интерпретация результатов

#### Прогнозирование на тестовом датасете

- Выполнить для тестового датасета те же этапы обработки и постронияния признаков
- Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете
- Прогнозы должны быть для всех примеров из тестового датасета (для всех строк)
- Соблюдать исходный порядок примеров из тестового датасета

### Обзор данных
##### Описание датасета

- Home Ownership - домовладение 
- Annual Income - годовой доход 
- Years in current job - количество лет на текущем месте работы 
- Tax Liens - налоговые обременения
- Number of Open Accounts - количество открытых счетов
- Years of Credit History - количество лет кредитной истории
- Maximum Open Credit - наибольший открытый кредит
- Number of Credit Problems - количество проблем с кредитом 
- Months since last delinquent - количество месяцев с последней просрочки платежа 
- Bankruptcies - банкротства 
- Purpose - цель кредита
- Term - срок кредита 
- Current Loan Amount - текущая сумма кредита
- Current Credit Balance - текущий кредитный баланс
- Monthly Debt - ежемесячный долг
- Credit Score - кредитный рейтинг  
- Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# Загрузка библиотек:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'svg'
matplotlib.rcParams.update({'font.size': 14})
%matplotlib inline

import pickle
import random
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm
import scipy.stats as stats


from sklearn.model_selection import train_test_split, ShuffleSplit, cross_val_score, learning_curve

# Алгоритмы кластеризации:

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

# Метрика F1:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

# Статистика

from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import probplot
from scipy.stats import chi2_contingency
from pathlib import Path

import statsmodels.api as sm
from statsmodels.formula.api import ols

### Загрузка данных

In [ ]:
TRAIN_DATASET_PATH = '/kaggle/input/654pds2courseproject/course_project_train.csv'
TEST_DATASET_PATH = '/kaggle/input/654pds2courseproject/course_project_test.csv'

df_train = pd.read_csv(TRAIN_DATASET_PATH)
df_test = pd.read_csv(TEST_DATASET_PATH)

### Обзор данных

- Home Ownership - домовладение
- Annual Income - годовой доход
- Years in current job - количество лет на текущем месте работы
- Tax Liens - налоговые обременения
- Number of Open Accounts - количество открытых счетов
- Years of Credit History - количество лет кредитной истории
- Maximum Open Credit - наибольший открытый кредит
- Number of Credit Problems - количество проблем с кредитом
- Months since last delinquent - количество месяцев с последней просрочки платежа
- Bankruptcies - банкротства
- Purpose - цель кредита
- Term - срок кредита
- Current Loan Amount - текущая сумма кредита
- Current Credit Balance - текущий кредитный баланс
- Monthly Debt - ежемесячный долг
- Credit Score - кредитный рейтинг 
- Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
df_train.head(10)

In [ ]:
# Корелляция с целевой переменной

corr_with_target = df_train.corr().iloc[:-1, -1].sort_values(ascending=False)
sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Корелляция с целевой переменной')
plt.show()

In [ ]:
df_train.shape

In [ ]:
df_train.info()

In [ ]:
df_test.describe().T

### Функции

In [ ]:
def find_cat(df_train):
    '''Функция находит все признаки, в которых первое значение строка'''
    for name in df_train.columns:
        s = ''
        s += name
        if (type(df_train[name][0]) == str):
            s += ' строка,'
        if (df_train[name].nunique()<=3):
            s += ' мало уникальных'
        if (s!=name):
            print (s)       
    
    
def mis_values_report(df_train):
    '''Функция считает и выводит пропущенные значения столбцов датафрейма'''
    mis_val = df_train.isnull().sum()
    perc_mis_val = 100 * df_train.isnull().sum() / len(df_train)
    report = pd.concat([mis_val, perc_mis_val], axis=1)
    report_columns = report.rename(columns = {0: 'Пропущенные значения', 1: '% от всех значений'})
    report_columns = report_columns[report_columns.iloc[:,1] != 0].sort_values('% от всех значений', ascending=False).round(1)
    print('Датафрейм имеет ' + str(df_train.shape[1]) + ' столбцов.\n' + str(report_columns.shape[0]) + ' столбцов имеют пропущенные значения')
    return report_columns

def hst(target_name):
    plt.figure(figsize=(20,8))
    ax1 = plt.subplot(121)
    ax1.set_xlabel(target_name)
    ax1.set_ylabel('Count')
    ax1.set_title(f'{target_name} distribution')
    df_test[target_name].hist()
    plt.subplot(122)
    probplot(df_test[target_name],dist='norm',plot=plt)
    plt.show()

In [ ]:
find_cat(df_train)

Выводы: Надо, по крайней мере, что-то сделать со столбцами, в которых записаны строковые значения. Также подозрения всегда вызывают столбцы с небольшим числом уникальных значений.

In [ ]:
mis_values_report(df_train)

### Целевая переменная

In [ ]:
TARGET_NAME = 'Credit Default'
BASE_FEATURE_NAMES = df_train.columns.drop(TARGET_NAME).tolist()
NUMB_FEATURE_NAMES = ['Annual Income','Tax Liens','Number of Open Accounts','Years of Credit History',
                     'Maximum Open Credit','Number of Credit Problems','Months since last delinquent',
                     'Bankruptcies','Current Loan Amount','Current Credit Balance','Monthly Debt','Credit Score']
CAT_FEATURE_NAMES = ['Years in current job','Home Ownership','Purpose','Term']

In [ ]:
y = df_train[[TARGET_NAME]]
y.info()

In [ ]:
# Добавляем поле ID - может пригодиться дальше 
df_train['ID'] = df_train.index.tolist()

In [ ]:
# Распределение целевой переменной

sns.countplot(x=TARGET_NAME, data=df_train)
plt.title('Распределение целевой переменной')
plt.show()


ВЫВОДЫ:



### Обработка данных

In [ ]:
# Обработка выбросов и пропусков
# Пропуски
df_train.isna().sum()

In [ ]:
df_test.isna().sum()


### Home Ownership - домовладение 

In [ ]:
df_train['Home Ownership'].value_counts()


In [ ]:
sns.countplot(x='Home Ownership', hue='Credit Default', data=df_train)
plt.show()

In [ ]:
df_train = pd.concat([df_train, pd.get_dummies(df_train['Home Ownership'], prefix='Home Ownership')], axis=1)
# Добавляем признаки в тестовый датасет
df_test = pd.concat([df_test, pd.get_dummies(df_test['Home Ownership'], prefix='Home Ownership')], axis=1)

### Annual Income - годовой доход 

In [ ]:
df_train['Annual Income'].describe()


In [ ]:
hst('Annual Income')

In [ ]:
# Добавляется новый бинарный признак о неизвестном доходе 1/0 (доход известен/доход не известен)
df_train['unknown_income'] = 0
df_train.loc[(df_train['Annual Income'].isnull()), 'unknown_income'] = 1

# В новую переменную annual_income_median записывается медианное значение дохода
annual_income_median = df_train['Annual Income'].median()

# Пропуски дохода заполняются медианным значением
df_train['Annual Income'].fillna(annual_income_median, inplace=True)

# Добавляем признаки в тестовый датасет
df_test['unknown_income'] = 0
annual_income_median = df_test['Annual Income'].median()
df_test.loc[(df_test['Annual Income'].isnull()), 'unknown_income'] = 1
df_test['Annual Income'].fillna(annual_income_median, inplace=True)

### Months since last delinquent - количество месяцев с последней просрочки кредита

In [ ]:
# Добавляется новый бинарный признак - 1/0 (задолженость/отсутствие задолженности)
df_train['has_delay'] = 1
df_train.loc[(df_train['Months since last delinquent'] == 0), 'has_delay'] = 0

# Заменяется пропуск NaN на 0 в 'Months since last delinquent'
df_train['Months since last delinquent'].fillna(0, inplace=True)

# Добавляем признаки в тестовый датасет
df_test['has_delay'] = 1
df_test['Months since last delinquent'].fillna(0, inplace=True)
df_test.loc[(df_test['Months since last delinquent'] == 0), 'has_delay'] = 0



In [ ]:
NEW_FEATURE_NAMES = ['has_delay','unknown_income']

In [ ]:
corr_with_target = df_train[BASE_FEATURE_NAMES + 
                            NEW_FEATURE_NAMES + 
                            [TARGET_NAME]].corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 8))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

In [ ]:
df_train.isna().sum()

### Years in current job - количество лет на текущем месте работы 

In [ ]:
df_train['Years in current job'].value_counts()

In [ ]:
sns.countplot(x="Years in current job", data=df_train)
plt.show()

In [ ]:
# посмотрим как выглядит распределение от стажа на месте
sns.set(style='darkgrid')
plt.figure(figsize=(10,6))
sns.countplot(df_train['Years in current job'], order = df_train['Years in current job'].value_counts().index)

In [ ]:
# Определение количества людей, в группах по стажу работы, не возвращающих кредит

g = sns.catplot("Credit Default", col="Years in current job", col_wrap=5,
                data=df_train,
                kind="count", height=3.5, aspect=.8, 
                palette='tab20')

#fig.suptitle('sf')
plt.show()

In [ ]:
# Уникальные значения Years in current job
unique_years_in_current_job = df_train['Years in current job'].unique()

var_experiance = unique_years_in_current_job[1:]

# NaN заменяется на рандомный опыт
df_train['Years in current job'].fillna(random.choice(var_experiance), inplace=True)

# Добавляем признаки в тестовый датасет

unique_years_in_current_job = df_test['Years in current job'].unique()
var_experiance = unique_years_in_current_job[1:]
df_test['Years in current job'].fillna(random.choice(var_experiance), inplace=True)

In [ ]:
df_train.isnull().sum()

### Credit Score - кредитный рейтинг

In [ ]:
df_train['Credit Score'].describe()


In [ ]:
hst('Credit Score')


In [ ]:
# Добавляется новая переменная unknown_credit_score по неизвестному Credit Score с присвоением 0 всем наблюдениям 
df_train['unknown_credit_score'] = 0

NEW_FEATURE_NAMES = ['has_delay','unknown_income','unknown_credit_score']

# В переменную credit_score_median записывается медианное значение рейтинга
credit_score_median = df_train['Credit Score'].median()

# По известному кредитному рейтингу присваивается 1 в переменной unknown_credit_score
df_train.loc[(df_train['Credit Score'].isnull()), 'unknown_credit_score'] = 1

# Пропуски рейтинга заполняются медианным значением
df_train['Credit Score'].fillna(credit_score_median, inplace=True)


# Добавляем признаки в тестовый датасет
df_test['unknown_credit_score'] = 0
NEW_FEATURE_NAMES = ['has_delay','unknown_income','unknown_credit_score']
credit_score_median = df_test['Credit Score'].median()
df_test.loc[(df_test['Credit Score'].isnull()), 'unknown_credit_score'] = 1
df_test['Credit Score'].fillna(credit_score_median, inplace=True)


In [ ]:
df_train.isnull().sum()

### Current Loan Amount - текущая сумма кредита

In [ ]:
df_train['Current Loan Amount'].value_counts()

In [ ]:
df_train['Current Loan Amount'].describe()

In [ ]:
hst('Current Loan Amount')



In [ ]:
# 99999999.0 в Current Loan Amount заменяются на медианные
median_current_loan_amount = df_train['Current Loan Amount'].median()
df_train.loc[(df_train['Current Loan Amount'] == 99999999.0), 'Current Loan Amount'] = median_current_loan_amount

# Добавляем признаки в тестовый датасет
median_current_loan_amount = df_test['Current Loan Amount'].median()
df_test.loc[(df_test['Current Loan Amount'] == 99999999.0), 'Current Loan Amount'] = median_current_loan_amount

### Bankruptcies - банкротства

In [ ]:
df_train['Bankruptcies'].value_counts()


In [ ]:
df_train['Bankruptcies'].fillna(0 , inplace=True)

In [ ]:
df_train.isnull().sum()


### Term - срок кредита

In [ ]:
df_train['Term'].value_counts()


In [ ]:
sns.countplot(x="Term", hue='Credit Default', data=df_train)
plt.show()

In [ ]:
df_train['Term'] = df_train['Term'].map({'Short Term':'1', 'Long Term':'0'}).astype(float)

# Добавляем признаки в тестовый датасет
df_test['Term'] = df_test['Term'].map({'Short Term':'1', 'Long Term':'0'}).astype(float)


In [ ]:
sns.countplot(x="Term", hue='Credit Default', data=df_train)
plt.show()

In [ ]:
def corr_(val, x1, x2):
    pay1_and_target_s = df_train.loc[df_train[val].isin([x1, x2]), ['ID', val, 'Credit Default']]#.sample(1000)
    table = pay1_and_target_s.pivot_table(values='ID', index=val, columns='Credit Default', aggfunc='count')
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    print(p)

In [ ]:
corr_('Term',0,1)

In [ ]:
corr_with_target = df_train[BASE_FEATURE_NAMES + 
                            NEW_FEATURE_NAMES + 
                            [TARGET_NAME]].corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 8))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

In [ ]:
df_train.isnull().sum()


### Number of Credit Problems - количество проблем с кредитом

In [ ]:
df_train['Number of Credit Problems'].value_counts()

### Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
df_train['Credit Default'].value_counts()


### Monthly Debt - ежемесячный долг

In [ ]:
df_train['Monthly Debt'].hist()


In [ ]:
df_test['Monthly Debt'].hist()


In [ ]:
df_train.loc[df_train['Monthly Debt'] > df_train['Monthly Debt'].quantile(.99), \
       ['Monthly Debt']] = df_train['Monthly Debt'].mean()

# Добавляем признаки в тестовый датасет
df_test.loc[df_test['Monthly Debt'] > df_test['Monthly Debt'].quantile(.99), \
       ['Monthly Debt']] = df_test['Monthly Debt'].mean()

In [ ]:
df_test['Monthly Debt'].hist()

### Current Credit Balance - текущий кредитный баланс

In [ ]:
df_train['Current Credit Balance'].describe()

In [ ]:
hst('Current Credit Balance')

In [ ]:
#считаем выбросы
df_train.loc[df_train['Current Credit Balance'] > df_train['Current Credit Balance'].quantile(.95), :].shape

In [ ]:
df_train.loc[df_train['Current Credit Balance'] > df_train['Current Credit Balance'].quantile(.95), \
       ['Current Credit Balance']] = df_train['Current Credit Balance'].quantile(.95) 
# Добавляем признаки в тестовый датасет
df_test.loc[df_test['Current Credit Balance'] > df_test['Current Credit Balance'].quantile(.95), \
       ['Current Credit Balance']] = df_test['Current Credit Balance'].quantile(.95) 

In [ ]:
df_test['Current Credit Balance'].hist()

### Purpose - цель кредита

In [ ]:
df_test['Purpose'].value_counts()


In [ ]:
df_test['Purpose'].hist()

In [ ]:
df_test['Purpose'].value_counts().sort_values(ascending=True).plot(kind='barh', title='Purpose', figsize=(10,6))

### Maximum Open Credit - наибольший открытый кредит

In [ ]:
df_train['Maximum Open Credit'].describe()

In [ ]:
hst('Maximum Open Credit')

In [ ]:
df_train.loc[df_train['Maximum Open Credit'] > df_train['Maximum Open Credit'].quantile(.95), ['Maximum Open Credit']] =\
            df_train['Maximum Open Credit'].quantile(.95)
# Добавляем признаки в тестовый датасет
df_test.loc[df_test['Maximum Open Credit'] > df_test['Maximum Open Credit'].quantile(.95), ['Maximum Open Credit']] =\
            df_test['Maximum Open Credit'].quantile(.95)

In [ ]:
hst('Maximum Open Credit')

### Years of Credit History - количество лет кредитной истории

In [ ]:
df_train['Years of Credit History'].value_counts()


In [ ]:
df_train['Years of Credit History'].hist()



— считаю нет необходимости в учетеб кредитной истории более 40 лет, это уже говорит о возрасте заемщика более 60 лет. Округлим все что больше 40 до 40.

In [ ]:
df_train.loc[df_train['Years of Credit History'] > 40, ['Years of Credit History']] = 40
# Добавляем признаки в тестовый датасет
df_test.loc[df_test['Years of Credit History'] > 40, ['Years of Credit History']] = 40

In [ ]:
df_train['Years of Credit History'].value_counts()

In [ ]:
df_train['Years of Credit History'].hist()


### Number of Open Accounts - количество открытых счетов

In [ ]:
hst('Number of Open Accounts')

In [ ]:
df_train.loc[df_train['Number of Open Accounts'] > df_train['Number of Open Accounts'].quantile(.95), ['Number of Open Accounts']] =\
            df_train['Number of Open Accounts'].median()
# Добавляем признаки в тестовый датасет
df_test.loc[df_test['Number of Open Accounts'] > df_test['Number of Open Accounts'].quantile(.95),  \
            ['Number of Open Accounts']] = df_test['Number of Open Accounts'].median()

In [ ]:
hst('Number of Open Accounts')

### Tax Liens - налоговые обременения

In [ ]:
df_train['Tax Liens'].value_counts()

### Что получилось

In [ ]:
df_train.info()

In [ ]:
def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('TRAIN\n\n' + classification_report(y_train_true, y_train_pred))
    print('TEST\n\n' + classification_report(y_test_true, y_test_pred))
    print('CONFUSION MATRIX\n')
    print(pd.crosstab(y_test_true, y_test_pred))

In [ ]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1) 

### Выбор модели

In [ ]:
df_train.head(10)

In [ ]:
NEW_FEATURE_NAMES

Пока пробую без признаков:
- 18  Home Ownership_Have Mortgage  7500 non-null   uint8  
- 19  Home Ownership_Home Mortgage  7500 non-null   uint8  
- 20  Home Ownership_Own Home       7500 non-null   uint8  
- 21  Home Ownership_Rent           7500 non-null   uint8  

In [ ]:
SELECTED_FEATURE_NAMES = NUMB_FEATURE_NAMES + NEW_FEATURE_NAMES


X = df_train[SELECTED_FEATURE_NAMES]
y = df_train[TARGET_NAME]

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.30, random_state=11)

### Нормализация

In [ ]:
scaler = StandardScaler()

df_norm = df_train.copy()
df_norm[NUMB_FEATURE_NAMES] = scaler.fit_transform(df_norm[NUMB_FEATURE_NAMES])

df_train = df_norm.copy()

### Балансировка

In [ ]:
df_for_balancing = pd.concat([X_train, y_train], axis=1)
df_balanced = balance_df_by_target(df_for_balancing, TARGET_NAME)
    
df_balanced[TARGET_NAME].value_counts()

In [ ]:
X_train = df_balanced.drop(columns=TARGET_NAME)
y_train = df_balanced[TARGET_NAME]

### CatBoost

In [ ]:
model_catb = catb.CatBoostClassifier(silent=True, random_state=42)
model_catb.fit(X_train, y_train)

y_train_pred = model_catb.predict(X_train)
y_test_pred = model_catb.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

In [ ]:
model_catb = catb.CatBoostClassifier(class_weights=[1, 3.5], silent=True, random_state=11)

In [ ]:
params = {'n_estimators':[50, 100, 200, 500, 700, 1000, 1200, 1500],
          'max_depth':[3, 5, 7]}

In [ ]:
cv=KFold(n_splits=3, random_state=42, shuffle=True)

In [ ]:
%%time

rs = RandomizedSearchCV(model_catb, params, scoring='f1', cv=cv, n_jobs=-1)
rs.fit(X, y)

In [ ]:
rs.best_params_

In [ ]:
rs.best_score_

In [ ]:
%%time

final_model = catb.CatBoostClassifier(n_estimators=500, max_depth=3,
                                      silent=True, random_state=42)
final_model.fit(X_train, y_train)

y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

get_classification_report(y_train, y_train_pred, y_test, y_test_pred)

### Подготовка тестового DS

In [ ]:
df_for_model = df_test[SELECTED_FEATURE_NAMES]
df_for_model.head(10)

In [ ]:
y_pred_output = final_model.predict(df_for_model)

In [ ]:
y_pred_output.shape


In [ ]:
result=pd.DataFrame({'Id':np.arange(2500), 'Credit Default': y_pred_output})


In [ ]:
RESULT_PATH='./KadnikovDV_CW_V3_predictions.csv'
result.to_csv(RESULT_PATH, index=False)

In [ ]:
SAMPLE = './KadnikovDV_CW_V3_predictions.csv'
df_sample = pd.read_csv(SAMPLE)

predictions = pd.DataFrame(index = df_sample['Id'])

predictions[TARGET_NAME] = y_pred_output
predictions